In [ ]:
'''
*Data Science- Round 2 (Final) Details*

Link:  https://www.kaggle.com/competitions/im-somewhat-of-a-cybersecurity-analyst-myself

1. The training data will be provided in a `CSV` format in the `training` directory
2. You’ll find 50 folds of the data in the `training/uploads/` directory.
    1. Here, you are expected to build the data loader using `pd.concat` and looping.
3. Once, you are done with this, you will be expected to build multiple models and train it using the training data generated from the previous step.
4. Next, you will be expected to get the predictions of the test data, in the expected format. The submission file contains:
    1. `machine_id`: An identifier to identify the machines
    2. `malware_status`: The output of the model (which is expected to be either 0 or 1).
5. Then just submit the above solution to Kaggle by converting the above to a csv format.
6. You will be ranked on a Public Leaderboard which is not a definite result as it will be tested on a portion of the test data.
7. The final ranks will be made available after the contests ends as you will expected to make you kernels/`.ipynb` files public on Kaggle.
    1. If this is not done, you WILL be disqualified even if you get the highest accuracy that your peers.
8. It is highly recommended to use the Kaggle kernels to build the models as you are free to use GPUs as well as TPUs as per your needs.

'''


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import plot_confusion_matrix


In [ ]:
'''
final submission csv file has two columns, machine_id and malware_status
with 3921483 rows and a header
this file should be generated after the model is trained and tested
'''

In [1]:

# Importing the dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


train1 = pd.read_parquet(
    '/Users/abdul/Desktop/Programming/HOD-CPP2nite/data/training_data/train1.parquet')
train2 = pd.read_parquet(
    '/Users/abdul/Desktop/Programming/HOD-CPP2nite/data/training_data/train2.parquet')
train3 = pd.read_parquet(
    '/Users/abdul/Desktop/Programming/HOD-CPP2nite/data/training_data/train3.parquet')
train = pd.concat([train1, train2, train3], axis=0)

test1 = pd.read_parquet(
    '/Users/abdul/Desktop/Programming/HOD-CPP2nite/data/testing_data/test1.parquet')
test2 = pd.read_parquet(
    '/Users/abdul/Desktop/Programming/HOD-CPP2nite/data/testing_data/test2.parquet')
test3 = pd.read_parquet(
    '/Users/abdul/Desktop/Programming/HOD-CPP2nite/data/testing_data/test3.parquet')
test = pd.concat([test1, test2, test3], axis=0)



In [2]:
print(train.shape)
print(test.shape)

(5950285, 83)
(3971198, 82)


In [3]:
# removing columns with 50% or more null values
train = train.dropna(thresh=len(train)*0.5, axis=1)
test = test.dropna(thresh=len(test)*0.5, axis=1)

In [13]:
print(train.shape)
print(test.shape)

(5950285, 76)
(3971198, 75)


In [14]:
print(train.columns)
print(test.columns)

Index(['security_product_name', 'machine_version', 'app_version',
       'av_sig_version', 'is_beta', 'rtp_state', 'is_sxs_passive', 'av_status',
       'av_prod_installed', 'av_prod_enabled', 'has_tpm', 'country_id',
       'city_id', 'org_id', 'geo_id', 'local_name_id', 'platform', 'processor',
       'os_version', 'os_build', 'os_suite', 'os_ptfm_sub_release',
       'os_build_lab', 'sku_edition', 'is_protected', 'auto_sample_opt_in',
       's_mode', 'ie_ver_id', 'smart_screen', 'firewall', 'uac_luaenable',
       'mdc2_form_factor', 'device_family', 'name_id', 'oem_model_id',
       'core_count_processer', 'manufacturer_id', 'model_id', 'disk_capacity',
       'disk_type', 'system_volume_capacity', 'has_optical_drive',
       'ram_capacity', 'chassis_type', 'diagonal_display_size',
       'display_resolution_horizontal', 'display_resolution_vertical',
       'power_profile', 'number_of_charges_for_battery', 'version',
       'architecture', 'branch', 'build_number', 'build_revisio

In [4]:
#Imputing na values with hot deck imputation
train = train.fillna(method='ffill')
test = test.fillna(method='ffill')


In [16]:
# Checking for null values
print(train.isnull().sum())
print(test.isnull().sum())


security_product_name    0
machine_version          0
app_version              0
av_sig_version           0
is_beta                  0
                        ..
aoac_support             0
is_gamer                 0
region_id                0
malware_status           0
machine_id               0
Length: 76, dtype: int64
security_product_name    0
machine_version          0
app_version              0
av_sig_version           0
is_beta                  0
                        ..
pen_support              0
aoac_support             0
is_gamer                 0
region_id                0
machine_id               0
Length: 75, dtype: int64


In [17]:
# Checking for duplicate values
print(train.duplicated().sum())
print(test.duplicated().sum())

0
0


In [5]:
#classifier models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

#model evaluation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import plot_confusion_matrix

#preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#pipelines
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

#feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel

#feature extraction
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import FastICA
from sklearn.decomposition import FactorAnalysis

#feature engineering
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

import pickle


In [6]:
# Splitting the dataset into the Training set and Test set
X = train.iloc[:, 1:-1].values
y = train.iloc[:, -1].values
X_test = test.iloc[:, 1:].values


In [ ]:

# Encoding categorical data
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)


In [7]:
# Splitting the dataset into the Training set and Test set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
""" sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)
 """
# model build 
for model in [RandomForestClassifier(), LogisticRegression(), SVC(), DecisionTreeClassifier(), KNeighborsClassifier(), GaussianNB(), GradientBoostingClassifier(), AdaBoostClassifier(), ExtraTreesClassifier(), BaggingClassifier()]:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    print(model)
    print(accuracy_score(y_val, y_pred))
    print(average_precision_score(y_val, y_pred))
    print('')
    pickle.dump(model, open('model{}.pkl'.format(model), 'wb'))

